# AI Impact on Jobs 2030
Predicting automation risk across global professions by 2030.

https://www.kaggle.com/datasets/khushikyad001/ai-impact-on-jobs-2030

In [ ]:
# Instalação de dependências
!pip install pandas openai python-dotenv kaggle -q
!pip install google-generativeai -q  # Opcional para Gemini

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import requests
import os
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Para exibir DataFrames de forma mais bonita
from IPython.display import display, Markdown

print("✅ Bibliotecas importadas com sucesso!")

# 1. EXTRACT - Extração dos Dados

In [ ]:
# Se não quiser usar Kaggle API, carregue o arquivo manualmente:
from google.colab import files
uploaded = files.upload()

# Ler o arquivo carregado
for filename in uploaded.keys():
    df = pd.read_csv(filename)
    print(f"Arquivo {filename} carregado com {len(df)} registros")

In [ ]:
# Função de extração
def extract_data(filepath='/content/AI_Impact_on_Jobs_2030.csv'):
    """
    Extrai dados do arquivo CSV
    """
    try:
        df = pd.read_csv(filepath)
        print(f"📊 Dados extraídos: {df.shape[0]} linhas, {df.shape[1]} colunas")
        print("\n📋 Colunas disponíveis:")
        for col in df.columns:
            print(f"  - {col}")
        return df
    except Exception as e:
        print(f"❌ Erro na extração: {e}")
        return None

# Executar extração
df_raw = extract_data()
if df_raw is not None:
    display(df_raw.head())

# 2. PREPARAÇÃO - Configuração da IA Generativa

In [ ]:
# Configurar OpenAI
from openai import OpenAI
import getpass

# Inserir a chave da API de forma segura
openai_api_key = getpass.getpass('🔑 Digite sua OpenAI API Key: ')

# Configurar cliente
client = OpenAI(api_key=openai_api_key)

# Testar conexão
try:
    test_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Teste de conexão. Responda com 'OK'"}],
        max_tokens=10
    )
    print("✅ OpenAI configurada com sucesso!")
except Exception as e:
    print(f"❌ Erro na configuração da OpenAI: {e}")

# 3. TRANSFORM - Transformação com IA

In [ ]:
# Função para limpar dados
def clean_data(df):
    """
    Limpa e prepara os dados
    """
    print("🧹 Iniciando limpeza dos dados...")

    # Criar cópia
    df_clean = df.copy()

    # Renomear colunas para facilitar
    df_clean.columns = [col.strip().replace(' ', '_').lower() for col in df_clean.columns]

    # Verificar valores nulos
    print(f"\n🔍 Valores nulos por coluna (se tiver contagem):")
    null_counts = df_clean.isnull().sum()
    for col, count in null_counts.items():
        if count > 0:
            print(f"  - {col}: {count} nulos ({count/len(df_clean)*100:.1f}%)")

    # Remover colunas com muitos valores nulos (opcional)
    threshold = 0.5  # 50% de valores nulos
    cols_to_drop = [col for col in df_clean.columns if df_clean[col].isnull().mean() > threshold]
    if cols_to_drop:
        print(f"\n🗑️  Removendo colunas com >{threshold*100}% nulos: {cols_to_drop}")
        df_clean = df_clean.drop(columns=cols_to_drop)

    # Identificar colunas de interesse
    print("\n🎯 Colunas identificadas:")
    job_cols = [col for col in df_clean.columns if 'job' in col or 'title' in col or 'occupation' in col]
    score_cols = [col for col in df_clean.columns if 'impact' in col or 'score' in col or 'probability' in col]

    print(f"  - Colunas de cargo: {job_cols if job_cols else 'Não encontradas'}")
    print(f"  - Colunas de score: {score_cols if score_cols else 'Não encontradas'}")

    return df_clean, job_cols, score_cols

# Executar limpeza
df_clean, job_cols, score_cols = clean_data(df_raw)
display(df_clean.head())

In [ ]:
# Função para gerar insights com IA


def generate_ai_insight(job_title, impact_score, provider="openai"):
    """
    Gera insights sobre o impacto da IA em um cargo específico
    """

    prompt = f"""
    Analise o impacto da Inteligência Artificial no cargo: {job_title}

    Contexto:
    - Score de impacto: {impact_score}/100 (quanto maior, maior o impacto)
    - Horizonte temporal: 2030
    - Setor: Tecnologia/Mercado de trabalho

    Por favor, forneça:
    1. Uma breve análise (2-3 frases) do impacto
    2. 3 habilidades que serão mais valorizadas
    3. Recomendação: "alta", "média" ou "baixa" prioridade para requalificação

    Formato a resposta como JSON:
    {{
        "analise": "texto da análise",
        "habilidades": ["hab1", "hab2", "hab3"],
        "prioridade": "alta/media/baixa",
        "tendencia": "automatizado/transformado/expandido"
    }}
    """

    try:
        if provider == "openai":
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Você é um especialista em futuro do trabalho e IA. Responda em português brasileiro."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=300
            )
            insight_text = response.choices[0].message.content

        elif provider == "gemini":
            response = gemini_model.generate_content(prompt)
            insight_text = response.text

        else:
            insight_text = '{"analise": "Insight não disponível", "habilidades": [], "prioridade": "media", "tendencia": "transformado"}'

        # Tentar extrair JSON da resposta
        try:
            # Encontrar texto entre chaves
            start = insight_text.find('{')
            end = insight_text.rfind('}') + 1
            json_str = insight_text[start:end]
            insight_json = json.loads(json_str)
        except:
            # Se não for JSON válido, criar estrutura padrão
            insight_json = {
                "analise": insight_text[:200] + "..." if len(insight_text) > 200 else insight_text,
                "habilidades": ["Análise de dados", "Pensamento crítico", "Adaptabilidade"],
                "prioridade": "média",
                "tendencia": "transformado"
            }

        return insight_json

    except Exception as e:
        print(f"⚠️  Erro ao gerar insight: {e}")
        return {
            "analise": f"Erro na análise: {str(e)[:100]}",
            "habilidades": [],
            "prioridade": "média",
            "tendencia": "transformado"
        }

In [ ]:
# Função principal de transformação
def transform_with_ai(df, job_cols, score_cols, sample_size=10, provider="openai"):
    """
    Adiciona insights gerados por IA ao DataFrame
    """
    print(f"🤖 Iniciando transformação com IA ({provider})...")
    print(f"📈 Processando {sample_size} amostras...")

    # Criar cópia
    df_transformed = df.copy()

    # Selecionar colunas para usar
    job_col = job_cols[0] if job_cols else df.columns[0]
    score_col = score_cols[0] if score_cols else df.columns[1] if len(df.columns) > 1 else df.columns[0]

    print(f"🔧 Usando colunas: '{job_col}' e '{score_col}'")

    # Processar amostra
    df_sample = df.head(sample_size).copy()

    insights_list = []
    habilidades_list = []
    prioridades_list = []
    tendencias_list = []

    # Barra de progresso
    from tqdm.notebook import tqdm

    for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="Gerando insights"):
        job_title = str(row[job_col])

        # Tentar converter score para numérico
        try:
            impact_score = float(row[score_col])
        except:
            impact_score = 50  # Valor padrão

        # Gerar insight
        insight = generate_ai_insight(job_title, impact_score, provider)

        insights_list.append(insight.get("analise", ""))
        habilidades_list.append(", ".join(insight.get("habilidades", [])))
        prioridades_list.append(insight.get("prioridade", "média"))
        tendencias_list.append(insight.get("tendencia", "transformado"))

        # Pausa para evitar rate limit
        time.sleep(1 if provider == "openai" else 0.5)

    # Adicionar colunas ao DataFrame
    df_sample['ai_insight'] = insights_list
    df_sample['habilidades_futuras'] = habilidades_list
    df_sample['prioridade_requalificacao'] = prioridades_list
    df_sample['tendencia_2030'] = tendencias_list
    df_sample['data_analise'] = datetime.now().strftime("%Y-%m-%d")

    print("✅ Transformação com IA concluída!")

    return df_sample

# Executar transformação
df_transformed = transform_with_ai(df_clean, job_cols, score_cols,
                                   sample_size=5, provider="openai")

# Mostrar resultados
display(df_transformed[[job_cols[0] if job_cols else df_clean.columns[0],
                       score_cols[0] if score_cols else df_clean.columns[1],
                       'ai_insight', 'habilidades_futuras']].head())

# 4. VISUALIZAÇÃO - Análise dos Resultados

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

# Configurar estilo dos gráficos
plt.style.use('default')
sns.set_palette("husl")

# Lista de stopwords em português para filtrar
STOPWORDS_PT = {
    'a','o','as','os','um','uma','uns','umas',
    'de','do','da','dos','das','em','no','na','nos','nas',
    'para','por','per','pra','pro','com','sem','sob','sobre','entre',
    'ao','à','às','aos','pelo','pela','pelos','pelas',
    'e','ou','mas','porque','pois','que','se','como','quando','onde','quanto',
    'este','esta','estes','estas','isso','isto','aquele','aquela','aqueles','aquelas',
    'eu','tu','ele','ela','nós','vos','eles','elas',
    'me','te','se','lhe','nos','vos','lhes',
    'meu','minha','meus','minhas',
    'teu','tua','teus','tuas',
    'seu','sua','seus','suas',
    'dela','dele','delas','deles',
    'quem','qual','quais','cujo','cuja','cujos','cujas',
    'todo','todos','toda','todas',
    'muito','muita','muitos','muitas',
    'pouco','pouca','poucos','poucas',
    'algum','alguma','alguns','algumas',
    'nenhum','nenhuma','nenhuns','nenhumas',
    'cada','mesmo','mesma','mesmos','mesmas',
    'também','já','ainda','só','não','sim'
}

# Adicionar mais stopwords relacionadas ao contexto
STOPWORDS_CONTEXT = {
    'ia','inteligência','artificial',
    'trabalho','emprego','ocupação','função',
    'tarefa','atividade','cargo','profissão',
    'automação','automatizado','automatizar',
    '20230','2030','tendência','transformado',
    'habilidades','futuras','impacto'
}

# Combinar todas as stopwords
ALL_STOPWORDS = STOPWORDS_PT.union(STOPWORDS_CONTEXT)

# Função para limpar texto e remover stopwords
def clean_text_for_wordcloud(text):
    """
    Limpa texto removendo stopwords e caracteres especiais
    """
    if not isinstance(text, str):
        return ""

    # Converter para minúsculas
    text = text.lower()

    # Remover caracteres especiais e números, manter apenas letras e espaços
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)

    # Remover stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in ALL_STOPWORDS and len(word) > 2]

    return ' '.join(filtered_words)

# Função para extrair palavras mais relevantes
def extract_keywords(text, top_n=20):
    """
    Extrai as palavras mais frequentes do texto limpo
    """
    clean_text = clean_text_for_wordcloud(text)
    words = clean_text.split()

    # Contar frequência
    word_counts = Counter(words)

    # Remover palavras muito curtas
    word_counts = {word: count for word, count in word_counts.items() if len(word) > 3}

    # Pegar as top_n palavras
    top_words = dict(word_counts.most_common(top_n))

    return top_words

In [ ]:
# Criar visualizações
fig, axes = plt.subplots(2, 2, figsize=(20, 10))

# 1. Distribuição de prioridades
if 'prioridade_requalificacao' in df_transformed.columns:
    priority_counts = df_transformed['prioridade_requalificacao'].value_counts()
    colors = ['#FF6B6B', '#FFD166', '#06D6A0']  # Vermelho, Amarelo, Verde
    priority_order = ['alta', 'média', 'baixa']

    # Reordenar para garantir a ordem correta
    priority_counts = priority_counts.reindex(priority_order, fill_value=0)

    bars = axes[0,0].bar(priority_counts.index, priority_counts.values, color=colors[:len(priority_counts)])
    axes[0,0].set_title('📊 Distribuição de Prioridade de Requalificação', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('Quantidade de Cargos', fontsize=12)
    axes[0,0].set_xlabel('Prioridade', fontsize=12)
    axes[0,0].tick_params(axis='x', rotation=45)

    # Adicionar valores nas barras
    for i, v in enumerate(priority_counts.values):
        axes[0,0].text(i, v + 0.1, str(v), ha='center', fontweight='bold')

    # Adicionar porcentagem
    total = priority_counts.sum()
    for i, v in enumerate(priority_counts.values):
        percentage = (v / total) * 100
        axes[0,0].text(i, v/2, f'{percentage:.1f}%', ha='center', va='center',
                      color='white', fontweight='bold', fontsize=11)

# 2. Distribuição de tendências
if 'tendencia_2030' in df_transformed.columns:
    trend_counts = df_transformed['tendencia_2030'].value_counts()
    trend_colors = ['#EF476F', '#118AB2', '#073B4C']  # Rosa, Azul, Azul escuro

    # Explodir a fatia mais relevante
    explode = [0.1 if i == trend_counts.idxmax() else 0 for i in trend_counts.index]

    wedges, texts, autotexts = axes[0,1].pie(trend_counts.values,
                                            labels=trend_counts.index,
                                            autopct='%1.1f%%',
                                            colors=trend_colors[:len(trend_counts)],
                                            explode=explode,
                                            startangle=90,
                                            shadow=True)

    axes[0,1].set_title('📈 Tendência dos Cargos em 2030', fontsize=14, fontweight='bold')

    # Melhorar a aparência dos textos
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(10)

# 3. Word Cloud ou Gráfico de Barras de Palavras-chave
try:
    # Tentar importar wordcloud
    from wordcloud import WordCloud

    # Combinar todos os insights
    all_insights = ' '.join(df_transformed['ai_insight'].astype(str))

    # Limpar texto
    clean_insights = clean_text_for_wordcloud(all_insights)

    if clean_insights.strip():  # Verificar se há texto limpo
        # Criar wordcloud
        wordcloud = WordCloud(width=500,
                             height=400,
                             background_color='white',
                             colormap='viridis',  # Mudar para outro colormap se quiser
                             max_words=50,
                             stopwords=set(),  # Já limpamos antes
                             contour_width=1,
                             contour_color='steelblue').generate(clean_insights)

        axes[1,0].imshow(wordcloud, interpolation='bilinear')
        axes[1,0].axis('off')
        axes[1,0].set_title('☁️  Palavras-chave mais Frequentes nos Insights',
                           fontsize=14, fontweight='bold', pad=20)

        # REMOVER OU COMENTAR ESTAS LINHAS PARA TIRAR O QUADRO:
        # axes[1,0].text(0.02, 0.98, 'Tamanho = Frequência\nCor = Relevância',
        #                transform=axes[1,0].transAxes,
        #                fontsize=9, verticalalignment='top',
        #                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    else:
        # Fallback: gráfico de barras das palavras mais frequentes
        top_words = extract_keywords(all_insights, top_n=15)

        if top_words:
            words = list(top_words.keys())
            counts = list(top_words.values())

            bars = axes[1,0].barh(range(len(words)), counts)
            axes[1,0].set_yticks(range(len(words)))
            axes[1,0].set_yticklabels(words, fontsize=10)
            axes[1,0].set_xlabel('Frequência')
            axes[1,0].set_title('📊 Palavras-chave mais Frequentes', fontsize=14, fontweight='bold')

            # Colorir barras baseado na frequência
            cmap = plt.cm.viridis
            for i, bar in enumerate(bars):
                normalized_value = counts[i] / max(counts)
                bar.set_color(cmap(normalized_value))
        else:
            axes[1,0].text(0.5, 0.5, 'Texto insuficiente\npara análise',
                          ha='center', va='center', fontsize=12)
            axes[1,0].set_title('Word Cloud não disponível', fontsize=14)

except ImportError:
    # Se wordcloud não estiver instalado, usar gráfico de barras
    all_insights = ' '.join(df_transformed['ai_insight'].astype(str))
    top_words = extract_keywords(all_insights, top_n=15)

    if top_words:
        words = list(top_words.keys())
        counts = list(top_words.values())

        bars = axes[1,0].barh(range(len(words)), counts, color='#118AB2')
        axes[1,0].set_yticks(range(len(words)))
        axes[1,0].set_yticklabels(words, fontsize=10)
        axes[1,0].set_xlabel('Frequência')
        axes[1,0].set_title('📊 Palavras-chave mais Frequentes (sem WordCloud)',
                           fontsize=14, fontweight='bold')

        # Adicionar valores nas barras
        for i, (word, count) in enumerate(zip(words, counts)):
            axes[1,0].text(count + 0.1, i, str(count), va='center', fontweight='bold')
    else:
        axes[1,0].text(0.5, 0.5, 'Instale: !pip install wordcloud',
                      ha='center', va='center', fontsize=12)
        axes[1,0].set_title('Word Cloud não instalado', fontsize=14)

# 4. Top habilidades do futuro
if 'habilidades_futuras' in df_transformed.columns:
    all_skills = []
    for skills in df_transformed['habilidades_futuras']:
        # Dividir por vírgula e limpar
        skill_list = [s.strip().lower() for s in str(skills).split(',')]
        all_skills.extend(skill_list)

    # Filtrar stopwords das habilidades
    all_skills = [skill for skill in all_skills
                  if skill not in ALL_STOPWORDS and len(skill) > 3]

    # Contar habilidades
    skill_counts = Counter(all_skills)

    # Pegar top 10 habilidades
    top_skills = skill_counts.most_common(10)

    if top_skills:
        skills_names = [skill[0].capitalize() for skill in top_skills]
        skills_counts = [skill[1] for skill in top_skills]

        # Criar gradiente de cores
        cmap = plt.cm.coolwarm
        colors = [cmap(i / len(skills_names)) for i in range(len(skills_names))]

        bars = axes[1,1].barh(range(len(skills_names)), skills_counts, color=colors)
        axes[1,1].set_yticks(range(len(skills_names)))
        axes[1,1].set_yticklabels(skills_names, fontsize=10)
        axes[1,1].set_xlabel('Frequência', fontsize=12)
        axes[1,1].set_title('💡 Top 10 Habilidades do Futuro',
                           fontsize=14, fontweight='bold', pad=15)

        # Adicionar valores nas barras
        for i, (name, count) in enumerate(zip(skills_names, skills_counts)):
            axes[1,1].text(count + 0.1, i, str(count), va='center',
                          fontweight='bold', fontsize=10)

        # Adicionar grade sutil
        axes[1,1].grid(axis='x', alpha=0.3, linestyle='--')
    else:
        axes[1,1].text(0.5, 0.5, 'Habilidades não\nencontradas',
                      ha='center', va='center', fontsize=12)
        axes[1,1].set_title('Habilidades do Futuro', fontsize=14)

# Ajustar layout
plt.suptitle('📈 Análise do Impacto da IA nos Empregos - 2030',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# 5. LOAD - Exportação dos Resultados

In [ ]:
def export_results(df, output_format='csv'):
    """
    Exporta os resultados transformados
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    if output_format == 'csv':
        filename = f'ai_jobs_analysis_{timestamp}.csv'
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"💾 CSV salvo como: {filename}")

        # Criar link para download
        from IPython.display import FileLink
        display(FileLink(filename))

    elif output_format == 'excel':
        filename = f'ai_jobs_analysis_{timestamp}.xlsx'
        df.to_excel(filename, index=False)
        print(f"💾 Excel salvo como: {filename}")
        display(FileLink(filename))

    elif output_format == 'json':
        filename = f'ai_jobs_analysis_{timestamp}.json'
        df.to_json(filename, orient='records', indent=2, force_ascii=False)
        print(f"💾 JSON salvo como: {filename}")
        display(FileLink(filename))

    return filename

# Exportar para múltiplos formatos
print("📤 Exportando resultados...")
csv_file = export_results(df_transformed, 'csv')
# excel_file = export_results(df_transformed, 'excel')

# 6. RESUMO EXECUTIVO COM IA

In [ ]:
# Gerar um resumo executivo dos resultados
def generate_executive_summary(df, provider="openai"):
    """
    Gera um resumo executivo com base na análise
    """

    # Preparar dados para o prompt
    if 'prioridade_requalificacao' in df.columns:
        high_priority = (df['prioridade_requalificacao'] == 'alta').sum()
        total_jobs = len(df)
        percent_high = (high_priority / total_jobs) * 100

    summary_prompt = f"""
    Com base na análise de {len(df)} cargos e seu impacto pela IA até 2030, gere um resumo executivo.

    Estatísticas principais:
    - Total de cargos analisados: {len(df)}
    - Cargos com alta prioridade de requalificação: {high_priority} ({percent_high:.1f}%)
    - Tendência predominante: {df['tendencia_2030'].mode()[0] if 'tendencia_2030' in df.columns else 'N/A'}

    Por favor, forneça um resumo em português brasileiro com:
    1. Visão geral dos impactos
    2. Recomendações estratégicas para RH/Governo
    3. 3 ações prioritárias

    Formato o texto de forma profissional para tomadores de decisão.
    """

    try:
        if provider == "openai":
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Você é um consultor estratégico de futuro do trabalho."},
                    {"role": "user", "content": summary_prompt}
                ],
                temperature=0.7,
                max_tokens=500
            )
            summary = response.choices[0].message.content

        elif provider == "gemini":
            response = gemini_model.generate_content(summary_prompt)
            summary = response.text
        else:
            summary = "Resumo executivo não disponível."

        return summary

    except Exception as e:
        return f"Erro ao gerar resumo: {str(e)}"

# Gerar e exibir resumo
print("📝 Gerando resumo executivo...")
executive_summary = generate_executive_summary(df_transformed, "openai")

display(Markdown("## 📊 RESUMO EXECUTIVO DA ANÁLISE"))
display(Markdown(executive_summary))